In [7]:
import numpy as np
import faiss
from functions import *
import pandas as pd

In [8]:
color_histograms_index = faiss.read_index("database/color_histograms.index") 
texture_histograms_index = faiss.read_index("database/histograms_texture.index")
hog_index = faiss.read_index("database/hog.index")
cnn_index = faiss.read_index("database/cnn.index")
embeddings_index = faiss.read_index("database/embeddings.index")

In [11]:
def precision_at_k(query_index, k, index, ground_truth):
    _, I = index.search(np.array([query_index]), k)
    return len(set(I[0]) & ground_truth) / k

In [10]:
paths_db = pd.read_csv("database/db.csv", index_col=0)
paths = paths_db['image'].values.tolist()
paths = ["images\\" + path for path in paths]

In [ ]:
def precision_evaluation(index, k):
    precision = 0
    for i in range(len(paths)):
        precision += precision_at_k(i, k, index, [i])
    return precision / len(paths)